[SIGN LANGUAGE DATASET](https://www.kaggle.com/datasets/nikhilgawai/sign-language-dataset)

In [ ]:
!wget -q https://github.com/DM-GY-9103-2024F-H/9103-utils/raw/main/src/data_utils.py
!wget -q https://github.com/DM-GY-9103-2024F-H/9103-utils/raw/main/src/image_utils.py

### Clean up and resize images

Export all as 3-channel `jpg`

In [ ]:
from os import listdir, path
from image_utils import open_image

In [ ]:
img_in_dir = "../9103-utils/datasets/image/sign-language"
img_out_dir = "./data/image/sign-language/"

in_dirs = sorted([d for d in listdir(img_in_dir) if path.isdir(path.join(img_in_dir, d))])

def is_image_file(f):
  f_lower = f.lower()
  for e in ["jpg", "jpeg", "png"]:
    if f_lower.endswith(e):
      return True
  return False

for d in in_dirs:
  in_path = path.join(img_in_dir, d)
  fnames = sorted([f for f in listdir(in_path) if is_image_file(f)])
  for i,f in enumerate(fnames):
    img = open_image(path.join(in_path, f)).convert("RGB")
    img.thumbnail([300, 200])
    img.save(path.join(img_out_dir, f"{d}-{('000'+str(i))[-2:]}.jpg"))

### Create DataLoaders

In [ ]:
import torch

from torch.utils.data import DataLoader
from torchvision.transforms import v2

from nn_utils import SignUtils

In [ ]:
train_dataset, test_dataset = SignUtils.train_test_split_cnn(test_ratio=0.2)
len(train_dataset), len(test_dataset)

In [ ]:
train_dl = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dl = DataLoader(test_dataset, batch_size=64, shuffle=False)

img,label = next(iter(train_dl))
print(img.shape, label.shape)
display(v2.ToPILImage()(img[0]))
display(v2.ToPILImage()(img[1]))
display(v2.ToPILImage()(img[2]))

with torch.no_grad():
  img,label = next(iter(test_dl))
  print(img.shape, label.shape)
  display(v2.ToPILImage()(img[0]))

### Create image lists for PCA

In [ ]:
import numpy as np
import pandas as pd
import os

from PIL import Image

In [ ]:
# Set the image path
data_dir = "data/image/sign-language"

In [ ]:
# load data
def load_images(data_dir):
  fnames = sorted([f for f in os.listdir(data_dir) if f.endswith("jpg")])

  data = []
  labels = []

  for img_name in fnames:
    label = img_name.split("-")[0]
    # Open the image and grayscale it
    img_path = os.path.join(data_dir, img_name)
    img = Image.open(img_path).convert('L')
    img = img.resize((64, 64))  # 调整到固定大小
    data.append(np.array(img).flatten())  # 展平图像
    labels.append(label)

  return np.array(data), np.array(labels), sorted(list(set(labels)))

In [ ]:
# load_images returns the categories now
X_np, y_np, categories = load_images(data_dir)

# this will have encode the categories into numbers
category2id = {c:i for i,c in enumerate(categories)}

In [ ]:
# Load and preprocess the data
X_np, y_np = load_images(data_dir, categories)
X = pd.DataFrame(X_np)
y = pd.DataFrame(y_np,columns=['label'])

# Code should be the same after this point